In [ ]:
!pip install langchain
!pip install jq
!pip install sentence-transformers
!pip install chromadb

In [3]:
from langchain_community.document_loaders import JSONLoader

In [ ]:

from langchain.docstore.document import Document
import json

def loadJSONFile(file_path):
    docs=[]
    # Load JSON file
    with open(file_path) as file:
        data = json.load(file)

    # Iterate through 'pages'
    for pair in data:
        msg = pair['msg']
        diff = pair['diff']
        lang = pair['language']
        snippets = pair['snippets']
        metadata={"title":pagetitle}

        # Process snippets for each page
        for snippet in snippets:
            index = snippet['index']
            childurl = snippet['childurl']
            text = snippet['text']
            docs.append(Document(page_content=text, metadata=metadata))
    return docs 

In [4]:
diff_loader = JSONLoader(
    file_path='../data/rag_data.json',
    jq_schema='.[].diff',
    text_content=False)

diff_data = diff_loader.load()

msg_loader = JSONLoader(
    file_path='../data/rag_data.json',
    jq_schema='.[].msg',
    text_content=False)

msg_data = msg_loader.load()

language_loader = JSONLoader(
    file_path='../data/rag_data.json',
    jq_schema='.[].language',
    text_content=False)

language_data = language_loader.load()


ImportError: jq package not found, please install it with `pip install jq`

## Import OpenAI model

In [ ]:
# 1-1000 data use java splitter, 1000-2000 use cpp splitter, 2000-3000 use csharp splitter, 3000-4000 use python splitter, 4000-5000 use javascript splitter
from langchain.text_splitter import Language
from langchain.text_splitter import RecursiveCharacterTextSplitter

languages = [Language.JAVA, Language.CPP, Language.CSHARP, Language.PYTHON, Language.JS]
splitters = [RecursiveCharacterTextSplitter.from_language(language, chunk_size=300, chunk_overlap=0) for language in languages]

data_split = []
for i, doc in enumerate(data):
    data_split += splitters[i // 1000].split_documents([doc])

In [ ]:
len(data_split)

27709

In [ ]:
data_split[0]

Document(page_content='diff --git a/MPChartLib/src/com/github/mikephil/charting/renderer/PieChartRenderer.java  b/MPChartLib/src/com/github/mikephil/charting/renderer/PieChartRenderer.java \npublic void drawValues(Canvas c){', metadata={'source': '/content/msg_nngen_nmt_codebert_chatgpt.json', 'seq_num': 1})

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
db = Chroma.from_documents(data_split, embeddings)

In [ ]:
type(db)

langchain_community.vectorstores.chroma.Chroma

In [ ]:
query = "diff --git a/java/client/src/org/openqa/selenium/support/events/EventFiringWebDriver.java  b/java/client/src/org/openqa/selenium/support/events/EventFiringWebDriver.java \npublic Timeouts timeouts(){} \npublic ImeHandler ime(){- throw new UnsupportedOperationException(\" Driver does not support IME interactions\");+ return options.ime();}@Beta \n"
docs = db.similarity_search_with_relevance_scores(query)

# print results
print(len(docs))

4


In [ ]:
documents = splitters[0].create_documents([query])

In [ ]:
docs = db.similarity_search_with_relevance_scores(documents[1].page_content)

In [ ]:
docs[3][0].metadata['seq_num']

173

1.0

In [ ]:
# Initialize an empty dictionary to store aggregate scores for each candidate ID
aggregate_scores = {}

# Iterate through each document in the documents list
for document in documents:
    # Apply similarity search function to the document
    results = db.similarity_search_with_relevance_scores(document.page_content)

    # Iterate through the results for each document
    for candidate_doc in results:
        id = candidate_doc[0].metadata['seq_num']
        score = candidate_doc[1]
        # Update the aggregate score for the candidate ID
        aggregate_scores[id] = aggregate_scores.get(id, 0) + score

# Find the candidate ID with the highest aggregate score
max_candidate_id = max(aggregate_scores, key=aggregate_scores.get)

# Output the candidate ID with the highest score
print("Candidate ID with the highest score:", max_candidate_id)
print("Score:", aggregate_scores.get(max_candidate_id))

Candidate ID with the highest score: 11
Score: 2.0
